In [ ]:
import pandas as pd
import folium
from sklearn.cluster import KMeans

df = pd.read_csv("data/accident_data.csv")
# Assume location_name → lat/lon mapping (define separately)
location_coords = {
    "Kranti Chowk": (19.8779, 75.3425),
    "CIDCO": (19.8893, 75.3208),
    "Jalna Road": (19.8645, 75.3504),
    "Seven Hills": (19.8658, 75.3299),
    "Aurangpura": (19.8843, 75.3250),
    "Railway Station": (19.8706, 75.3423)
}
df["lat"] = df["location_name"].map(lambda x: location_coords.get(x, (None, None))[0])
df["lon"] = df["location_name"].map(lambda x: location_coords.get(x, (None, None))[1])

kmeans = KMeans(n_clusters=3)
df_cluster = df.dropna(subset=["lat", "lon"])
df_cluster["cluster"] = kmeans.fit_predict(df_cluster[["lat", "lon"]])

m = folium.Map(location=[19.8762, 75.3433], zoom_start=13)
for _, row in df_cluster.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        color="red" if row["cluster"]==0 else "blue",
        fill=True,
    ).add_to(m)

m.save("report/hotspot_map.html")
